## AA-ICP: Iterative Closest Point with Anderson Acceleration

Main idea:
 * ICP uses iterative process, each iteration returns a transformation.
 * Using AA on these transformations may improve convergence.
 * ICP is considered a blackbox algorithm.

Reading some 3D model:

In [2]:
# Read model

# Visualize model


We perform a random transformation of this model:

In [3]:
# Do random transformation - shift and rotation.
#  Shift might need to be small.
#  Store transformation for future comparison

#Visualize new model


Now lets compare basic ICP algorithm with improved one:

In [1]:
# Do ICP

# Do ICP_AA

# Compare results

# Compare time

# Plot convergence

In [ ]:
from functools import partial
from scipy.optimize import root
from anderson import anderson
from picard import picard

# Signature is equivalent to scipy.optimize.root: root(F, x0, args, kwargs)


maxiter = 100
iterative_methods = {
    "Picard iterations" : picard,
    "Anderson" : anderson,
    "Anderson, limited alpha" : partial(anderson, alpha_limit=10.0),
    "Anderson, positive alpha0" : partial(anderson, alpha0_min=0.000),
    "Scipy's AA, default" : (lambda F, x0, maxiter, *args, **kwargs: root((lambda x: F(x) - x), x0, method="anderson", options = {"nit": maxiter}, *args, **kwargs).x),
#    "Scipy's AA, M=10" : (lambda F, x0, maxiter: root((lambda x: F(x) - x), x0, method="anderson", options = {"nit": maxiter, "jac_options":{"M":10}}).x),
#    "Scipy's AA, M=10, line_search=None" : (lambda F, x0, maxiter: root((lambda x: F(x) - x), x0, method="anderson", options = {"nit": maxiter, "line_search": None, "jac_options":{"M":10}}).x),
#    "Scipy's AA, M=10, line_search=None, w0" : (lambda F, x0, maxiter: root((lambda x: F(x) - x), x0, method="anderson", options = {"nit": maxiter, "line_search": None, "jac_options":{"M":10,"w0":0.0001}}).x),
}

In [ ]:
import matplotlib.pyplot as plt
import numpy

# Function
def F(x):
    root = numpy.array(range(x.size))
    return (root + (x - root)/(1+abs(x - root)))
# Starting point
x0 = numpy.zeros(10)
# Root
x_star = picard(F, x0, maxiter = 10000)
# How to calculate residual for the solution
def residual(x):
    #x_star = -np.array(range(x.size))
    return numpy.linalg.norm(x - x_star)

plt.rcParams["figure.figsize"] = [16,9]

for name, method in iterative_methods.items():
    print("Current method : %s" % name)
    xs = [residual(x0)]
    callback = (lambda x,r,xs=xs : xs.append(residual(x)))
    xs += [residual(method(F, x0, maxiter = maxiter, callback = callback))]
    plt.plot(xs, label = name)
plt.title("Convergence of different methods")
plt.xlabel("Iterations")
plt.ylabel("Residue")
#plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.show()